该文件主要是为了生成需要的数据，并将数据保存到 data.db 中

In [2]:
import os
import string
import datetime
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()),"data.db")
engine_path = "sqlite:///" + db_path
# 创建数据库引擎
engine = create_engine(engine_path)

### 用户商品交易数据
> 适用于RFM模型
>
>user_id（用户id）、create_time（创建时间）、order_id（商品id）、amount（交易金额）

In [9]:
sale_df = pd.read_csv("./sale.csv")

In [10]:
sale_df.shape

(76050, 4)

In [12]:
# 替换时间为 2020 年
sale_df["create_time"] = sale_df["create_time"].map(
    lambda x:x.replace("2018","2021") if pd.notnull(x) else x
)

In [13]:
sale_df.sample(5)

,user_id,create_time,order_id,amount
2251,116296,2021/1/10,4190926070,3444.0
10055,148501,2021/2/13,4194568993,89.0
37100,141044,2021/6/17,4197665374,771.0
24916,131141,2021/4/20,4195456200,15.0
43877,162531,2021/7/20,4199239372,2244.0


In [14]:
sale_df.to_sql("business", engine, if_exists='replace', index=True)

### 用户数据
>与comment表连用可以用于舆情分析
>
> 保存到数据库 users 表中
>
> user_id（用户id）、username（用户名）、age（年龄）
>

In [17]:
name_str_list = [i for i in string.ascii_letters+string.digits]

In [18]:
# 随机生成10000个用户的信息
user_df = pd.DataFrame(data={
    "user_id":range(1,10001),
    "username":["".join(np.random.choice(name_str_list,6)) for _ in range(10000)],
    "age":np.random.choice(range(20,50),10000),
})

In [19]:
user_df.sample(2)

,user_id,username,age
8722,8723,W58rqq,23
4944,4945,VD9D3G,21


In [20]:
user_df.to_sql("users", engine, if_exists='replace', index=False)

### 商品评价数据
> 商品评论数据来自 [阿里云天池比赛](https://tianchi.aliyun.com/competition/entrance/531890/introduction)
>
> 数据保存到数据库 comment 表中
>
>

In [21]:
comment_df = pd.read_csv("./earphone_sentiment.csv")

In [22]:
comment_df.sample(2)

,content_id,content,subject,sentiment_word,sentiment_value
2141,2152,正经的数播都会用音频优化过的时钟，这对数字音源最后的声场准确和声音定位都是至关重要的，普通p...,其他,NaN,0
9850,9877,这个包装和做工看上去挺值的，耳罩看得出来下了功夫。官方有没有测过频响？,外形,NaN,0


In [23]:
comment_df.shape

(17176, 5)

In [24]:
# 添加创建时间
index = pd.date_range("2021-01-01", "2021-12-31")

# 随机选择 17176 条时间
comment_df["create_time"] = np.random.choice(a=index,size=17176)

# 添加用户id
comment_df["user_id"] = range(1,len(comment_df)+1)

In [25]:
comment_df.head(2)

,content_id,content,subject,sentiment_word,sentiment_value,create_time,user_id
0,0,Silent Angel期待您的光临，共赏美好的声音！,其他,好,1,2021-03-02,1
1,2,这只HD650在1k的失真左声道是右声道的6倍左右，也超出官方规格参数范围（0.05%），看...,其他,NaN,0,2021-06-24,2


In [26]:
comment_df.to_sql("comment", engine, if_exists='replace', index=False)

### 观研报告网数据
> 观研报告网：http://data.chinabaogao.com/ 
>
> read_html() 官网地址：[点击跳转](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html?highlight=read_html)
>
> 网页中有表格元素（tbody）均可以下载  


In [27]:
# 举例说明，该 url 下有两个表格
# 其他页面数据，只需修改该 url 
url = "http://data.chinabaogao.com/dianxin/2021/11305620U2021.html"

result = pd.read_html(io=url,header=0)

for d in result:
    display(d)

,时间,全国居民平均每百户年末拥有量（部）,城镇居民平均每百户年末拥有量（部）,农村居民平均每百户年末拥有量（部）
0,2014年,215.9,216.6,215.0
1,2015年,224.8,223.8,226.1
2,2016年,235.4,231.4,240.7
3,2017年,240.0,235.4,246.1
4,2018年,249.1,243.1,257.0
5,2019年,253.2,247.4,261.2
6,2020年,253.8,248.7,260.9


,省市,居民平均每百户年末拥有量（部）,城镇居民平均每百户年末拥有量（部）,农村居民平均每百户年末拥有量（部）
0,北京市,234.5,232.2,251.1
1,天津市,235.8,236.4,232.6
2,河北省,244.6,239.4,250.9
3,山西省,226.1,239.4,211.9
4,内蒙古自治区,233.5,231.6,236.3
5,辽宁省,211.8,208.1,219.6
6,吉林省,237.1,231.1,245.7
7,黑龙江省,229.1,223.9,236.8
8,上海市,226.0,228.3,207.0
9,江苏省,252.3,254.2,248.6
